In [53]:
%matplotlib inline
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import pandas as pd
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter

In [54]:
# Load CSV data into a pandas DataFrame
df = pd.read_csv('data/Battery_RUL_cleaned.csv')
df = df[df.columns[1:]]  # Remove the first column

# Last column is the target variable
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# Convert to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

# Create a TensorDataset
dataset = TensorDataset(X_tensor, y_tensor)

# Split the dataset into training and testing sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Create data loaders
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [55]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define models
class NeuralNetwork(nn.Module):
    def __init__(self, h_size):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(7, h_size),
            nn.ReLU(),
            nn.Linear(h_size, h_size//2),
            nn.ReLU(),
            nn.Linear(h_size//2, h_size//4),
            nn.ReLU(),
            nn.Linear(h_size//4, 1)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
class SimpleNN(nn.Module):
    def __init__(self, h_size):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(7, h_size),
            nn.ReLU(),
            nn.Linear(h_size, 1)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
class NN(nn.Module):
    def __init__(self, h_size, d_rate):
        super().__init__()
        self.Flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(7, h_size),
            nn.BatchNorm1d(h_size),
            nn.ReLU(),
            nn.Dropout(d_rate),
            nn.Linear(h_size, h_size//2),
            nn.BatchNorm1d(h_size//2),
            nn.ReLU(),
            nn.Linear(h_size//2, h_size//4),
            nn.BatchNorm1d(h_size//4),
            nn.ReLU(),
            nn.Linear(h_size//4, 1)
        )

    def forward(self, x):
        x = self.Flatten(x)
        logit = self.linear_relu_stack(x)
        return logit

class LSTMNN(nn.Module):
    def __init__(self, hidden_size, num_layers, dropout_rate):
        super().__init__()
        self.lstm = nn.LSTM(7, hidden_size, num_layers, batch_first=True, dropout=dropout_rate) # nn.GRU for Gated Recurrent Unit
        self.fc = nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        x = x.unsqueeze(1)
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # Take the output at the last timestep
        return out

class CNN(nn.Module):
    def __init__(self, num_filters, kernel_size):
        super().__init__()
        self.conv1 = nn.Conv1d(7, num_filters, kernel_size)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(2)
        self.fc = nn.Linear(num_filters, 1)
    
    def forward(self, x):
        x = x.permute(0, 2, 1)  # Change shape to (batch, channels, features)
        x = self.pool(self.relu(self.conv1(x)))
        x = torch.flatten(x, start_dim=1)
        x = self.fc(x)
        return x





#model = SimpleNN(10).to(device)
#model = NN(512, 0.2).to(device)
#model = NeuralNetwork(512).to(device)
model = LSTMNN(32, 1, 0).to(device)
#model = CNN(16, 3).to(device)
print(model)

Using cpu device
LSTMNN(
  (lstm): LSTM(7, 32, batch_first=True)
  (fc): Linear(in_features=32, out_features=1, bias=True)
)


In [56]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

In [57]:
def train(dataloader, model, loss1, loss2, optimizer, epoch):
    model.train()
    for batch in dataloader:
        X, y = batch
        X, y = X.to(device), y.to(device)
        # Compute prediction error
        pred = model(X)
        loss = 0.7 * loss1(pred.squeeze(), y) + 0.3 * loss2(pred.squeeze(), y)
        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # Log the loss
        writer.add_scalar('Training Loss', loss.item(), epoch)
        return loss


In [58]:
def test(dataloader, model, loss1, loss2, epoch):
    num_batches = len(dataloader)
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for batch in dataloader:
            X, y = batch
            X, y = X.to(device), y.to(device)
            pred = model(X).squeeze()
            test_loss += loss1(pred, y).item() + loss2(pred, y).item()
    test_loss /= num_batches

    # Log the loss
    writer.add_scalar('Test Loss', test_loss, epoch)
    return test_loss


In [59]:
# Initialize TensorBoard writer
writer = SummaryWriter('runs/experiment_LSTM')

loss1 = nn.MSELoss()
loss2 = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-1)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.8)
epochs = 1500

# Initialize early stopping
early_stopping = EarlyStopping(patience=50, min_delta=0.01)

pbar = tqdm(range(epochs))
for t in pbar:
    train_loss = train(train_dataloader, model, loss1, loss2, optimizer, t)
    test_loss = test(test_dataloader, model, loss1, loss2, t)
    test_loss_MAE = test(test_dataloader, model, nn.L1Loss(), nn.L1Loss(), t)
    test_loss_MSE = test(test_dataloader, model, nn.MSELoss(), nn.MSELoss(), t)
    scheduler.step()
    pbar.set_description(f"Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Test Loss MAE: {test_loss_MAE:.4f}, Test Loss MSE: {test_loss_MSE:.4f}")

    # Check for early stopping
    #early_stopping(test_loss)
    if early_stopping.early_stop:
        print("Early stopping")
        break

print("Done!")
torch.save(model.state_dict(), 'models/model.pt')  # Save only the state_dict

# Close the TensorBoard writer
writer.close()

model.eval()
X, y = next(iter(test_dataloader))
X, y = X.to(device), y.to(device)
with torch.no_grad():
    pred = model(X)
    predicted, actual = pred[0], y[0]
    print(f'Input: "{X[0]}", Predicted: "{predicted}", Actual: "{actual}"')

Train Loss: 341633.4062, Test Loss: 833062.7398:   1%|          | 12/1500 [00:00<01:41, 14.62it/s]


KeyboardInterrupt: 